# Sensor placement

[Simon Dobson](mailto:simon.dobson@st-andrews.ac.uk) <br>
School of Computer Science, University of St Andrews, Scotland UK

In [1]:
import json
import pickle
from datetime import datetime, timedelta
from itertools import product, combinations
from functools import reduce
import numpy
import netCDF4 as nc
from pandas import DataFrame
from geopandas import GeoDataFrame, GeoSeries, read_file
from networkx import Graph

import sensor_placement
from sensor_placement import *

from pyproj import CRS, Transformer, Geod
from geovoronoi import voronoi_regions_from_coords, points_to_coords
import folium
import folium.plugins
import shapely
from shapely.geometry import Point, Polygon, MultiPolygon, shape
from shapely.ops import cascaded_union

### Import the CEH-GEAR dataset

The CEH_GEAR dataset is the "reference" datase for rainfall interpolated at 1km squares across Great Britain. 

In [2]:
days_base = datetime(year=1800, month=1, day=1)

In [3]:
def daysToDate(days):
    '''Return the date of the given day offset'''
    return days_base + timedelta(days=days) 

def dateToDays(date):
    '''Return the days offset to the given date.'''
    return date - days_base

def dateToIndex(ds, date):
    '''Return the index into the given dataset of the given date.'''
    return dateToDays(date).days - ds['time'][0]

In [4]:
ceh_filename='datasets/CEH_GEAR_monthly_GB_2017.nc'
ceh = nc.Dataset(ceh_filename)

In [5]:
xs, ys, ts, lats, lons = ceh['x'][:], ceh['y'][:], ceh['time'][:], ceh['lat'][:], ceh['lon'][:]
rainfall = ceh['rainfall_amount'][1, :, :]
distance = ceh['min_dist'][1, :, :]

In [6]:
print(daysToDate(min(ts)))

2017-01-01 00:00:00


The data is referenced by co-ordinates on the UK National Grid. Since we need to be able to also work in standard latitude-longitudfe co-ordinates, we construct a transformer between the two co-ordinate reference systems.  

In [2]:
uk_grid_crs = CRS.from_string('EPSG:27700')   # UK national grid
latlon_crs = CRS.from_string('EPSG:4326')     # global Mercator (WGS 84)

proj = Transformer.from_crs(uk_grid_crs, latlon_crs)
proj_inv = Transformer.from_crs(latlon_crs, uk_grid_crs)

### Import the boundaries of counties

We construct a dataframe with the names and b oundary shapoes of counties (and other administrative areas of the UK) by reading the master shape file.

In [3]:
boundaries_filename = 'datasets/UK_BUC.geojson'
with open(boundaries_filename, 'r') as fh:
    counties_json = json.load(fh)

In [4]:
counties = GeoDataFrame(columns=['county', 'geometry'])
for c in counties_json['features']:
    counties.loc[len(counties.index)] = {'county': c['properties']['ctyua18nm'],
                                         'geometry': shape(c['geometry'])}

We'll focus on Fife for the rest of this notebook.

In [10]:
fife = counties[counties['county'] == 'Fife'].iloc[0]

We extract the boundary polygon for Fife.

In [11]:
fife_boundary = fife['geometry']

We also extract the co-ordinates for the "centre" of Fife, simply for poisitioning the maps.

In [12]:
mid_fife = list(list(fife_boundary.centroid.coords)[0])
mid_fife.reverse()
mid_fife

[56.22895222627837, -3.1263301904853984]

### Load the SEPA data

In [13]:
sepa_filename='datasets/sepa_monthly_2017.nc'
sepa = nc.Dataset(sepa_filename)

In [14]:
sepastations = GeoDataFrame(columns=['name', 'id', 'longitude', 'latitude', 'geometry'])
for i in range(len(sepa['station'])):
    lat, lon = proj.transform(float(sepa['x'][i]), float(sepa['y'][i]))
    sepastations.loc[i] = {'id': int(sepa['station'][i]),
                           'name': sepa['name'][i],
                           'longitude': lon,
                           'latitude': lat,
                           'geometry': Point(lon, lat)}
sepastations.set_index('id', inplace=True)

In [15]:
fife_stations = sepastations[sepastations['geometry'].within(fife_boundary)]

In [16]:
fife_stations

,name,longitude,latitude,geometry
id,,,,
15198,Annfield,-3.382615,56.059090,POINT (-3.38262 56.05909)
501947,Baintown Rain-gauge,-3.049613,56.215117,POINT (-3.04961 56.21512)
338380,Cambo Sands,-2.664088,56.298649,POINT (-2.66409 56.29865)
15151,Fife Airport,-3.225782,56.177556,POINT (-3.22578 56.17756)
473550,Kinghorn Ecology Centre Rain Gauge,-3.206299,56.069925,POINT (-3.20630 56.06992)
335617,Kirkcaldy,-3.159471,56.115300,POINT (-3.15947 56.11530)
15083,Newton of Falkland,-3.195810,56.249730,POINT (-3.19581 56.24973)
15070,Rossie Farm,-3.213366,56.294485,POINT (-3.21337 56.29448)
15155,Saline,-3.593886,56.119442,POINT (-3.59389 56.11944)


### Displaying the reference data 

To display the rainfall data we extract from the CEH-GEAR dataset all the points that lie within the boundary of interest.

In [334]:
fife_points = GeoDataFrame(columns=['i_east', 'i_north', 'longitude', 'latitude', 'geometry'])
for i in range(len(xs)):
    for j in range(len(ys)):
        if not rainfall.mask[j, i]:
            lat, lon = proj.transform(xs[i], ys[j])
            p = Point(lon, lat)
            if fife_boundary.contains(p):
                fife_points.loc[len(fife_points.index)] = {'i_east': i,
                                                           'i_north': j,
                                                           'longitude': lon,
                                                           'latitude': lat,
                                                           'geometry': p}

In [335]:
fife_points_rainfall = fife_points.copy()
fife_points_rainfall['rainfall'] = fife_points_rainfall.apply(lambda r: rainfall[r['i_north'], r['i_east']], axis=1)

We can then construct a map with overlays for the region of interest and the interpolated rainfall.

In [336]:
uk_ceh = folium.Map(location=mid_fife, tiles="Stamen Terrain", zoom_start=10)

# add the boundary of Fife
folium.GeoJson(fife['geometry']).add_to(uk_ceh)

# add the stations
for i in range(len(fife_stations)):
    s = fife_stations.iloc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W)').add_to(uk_ceh)

# add the heat map
_ = folium.plugins.HeatMap(data=fife_points_rainfall[['latitude', 'longitude', 'rainfall']], min_opacity=0.01, radius=40, blur=40).add_to(uk_ceh)

In [337]:
uk_ceh

Alternatively, if the actual observations at the stations themselves are of interst, we can plot them too. We first load the raw data.

In [342]:
ss = fife_stations.index
sis = list(map(lambda id:list(sepa['station'][:]).index(id), ss))
fis = sepa['rainfall_amount'][1, sis]
print('Rainfall {:.02f}--{:.02f}mm'.format(min(fis), max(fis)))

Rainfall 52.20--95.60mm


In [344]:
fife_stations

,name,longitude,latitude,geometry
id,,,,
15198,Annfield,-3.382615,56.059090,POINT (-3.38262 56.05909)
501947,Baintown Rain-gauge,-3.049613,56.215117,POINT (-3.04961 56.21512)
338380,Cambo Sands,-2.664088,56.298649,POINT (-2.66409 56.29865)
15151,Fife Airport,-3.225782,56.177556,POINT (-3.22578 56.17756)
473550,Kinghorn Ecology Centre Rain Gauge,-3.206299,56.069925,POINT (-3.20630 56.06992)
335617,Kirkcaldy,-3.159471,56.115300,POINT (-3.15947 56.11530)
15083,Newton of Falkland,-3.195810,56.249730,POINT (-3.19581 56.24973)
15070,Rossie Farm,-3.213366,56.294485,POINT (-3.21337 56.29448)
15155,Saline,-3.593886,56.119442,POINT (-3.59389 56.11944)


In [347]:
uk_stations = folium.Map(location=mid_fife, tiles="Stamen Terrain", zoom_start=10)
folium.GeoJson(fife['geometry']).add_to(uk_stations)

# add the station obsertaions
for i in fife_stations.index:
    s = fife_stations.loc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    j = list(sepa['station'][:]).index(i)
    rain = sepa['rainfall_amount'][1, j]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} {rain:.2f}mm').add_to(uk_stations)

In [348]:
uk_stations

### Building a network of stations

We might also want to construct a neighbourhood graph from the stations. For this we assume that stations are the nodes of a network, with edges between nodes that are within a certain distance of each other.

In [351]:
g_fife = Graph()

# construct nodes for each station
for i in fife_stations.index:
    s = fife_stations.loc[i]
    g_fife.add_node(i)
    g_fife.nodes[i]['name'] = s['name']
    g_fife.nodes[i]['id'] = i
    g_fife.nodes[i]['longitude'] = s['longitude']
    g_fife.nodes[i]['latitude'] = s['latitude']

# form edges between nodes within 25km of each other
geodesic = Geod(ellps='clrk66')
for n, m in combinations(list(g_fife.nodes()), 2):
    lon1, lat1 = g_fife.nodes[n]['longitude'], g_fife.nodes[n]['latitude']
    lon2, lat2 = g_fife.nodes[m]['longitude'], g_fife.nodes[m]['latitude']
    _, _, d = geodesic.inv(lon1, lat1, lon2, lat2)
    if d <= 25000:
        g_fife.add_edge(n, m)
        g_fife.edges[n, m]['distance'] = d

In [352]:
fife_stations_network = folium.Map(location=mid_fife, tiles="Stamen Terrain", zoom_start=10)
folium.GeoJson(fife['geometry']).add_to(fife_stations_network)

# add the stations
for n in g_fife.nodes():
    name, id, lat, lon = g_fife.nodes[n]['name'], g_fife.nodes[n]['id'], g_fife.nodes[n]['latitude'], g_fife.nodes[n]['longitude']
    i = list(sepa['station'][:]).index(id)
    rain = sepa['rainfall_amount'][1, i]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W) {rain:.2f}mm').add_to(fife_stations_network)
        
# add the edges
for n, m in g_fife.edges():
    ps = [(g_fife.nodes[n]['latitude'], g_fife.nodes[n]['longitude']), (g_fife.nodes[m]['latitude'], g_fife.nodes[m]['longitude'])]
    tip = 'Distance {d:.2f}km'.format(d=g_fife.edges[n, m]['distance'] / 1000)
    folium.PolyLine(ps, color='red', tooltip=tip, weight=2, opacity=1).add_to(fife_stations_network)

In [353]:
fife_stations_network

### Constructing the Voronoi cells

Interpolation works by mapping observed values through the Voronoi cells (also known as the Thiessen polygon in geographical systems).

In [17]:
stations = fife_stations[['name', 'geometry']]
stations.crs = 'EPSG:4326'

boundary = GeoDataFrame(columns=['geometry'])
boundary.loc[len(boundary.index)] = {'geometry': fife['geometry']}
boundary.crs = 'EPSG:4326'

cell_centres = points_to_coords(stations['geometry'])
boundary_shape = cascaded_union(boundary['geometry'])

In [355]:
voronoi_cells, voronoi_centres = voronoi_regions_from_coords(cell_centres, boundary_shape)

In [356]:
fife_cells = GeoDataFrame(columns=['geometry'])
for c in voronoi_cells.values():
    fife_cells.loc[len(fife_cells.index)] = {'geometry': c}

In [358]:
fife_stations_voronoi = folium.Map(location=mid_fife, tiles="Stamen Terrain", zoom_start=10)
folium.GeoJson(fife['geometry']).add_to(fife_stations_voronoi)

# add the stations
for i in fife_stations.index:
    s = fife_stations.loc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    j = list(sepa['station'][:]).index(i)
    rain = sepa['rainfall_amount'][1, j]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W) {rain:.2f}mm').add_to(fife_stations_voronoi)
    
# add the Voronoi cells
for c in fife_cells['geometry']:
    folium.GeoJson(c).add_to(fife_stations_voronoi)

In [359]:
fife_stations_voronoi

### Constructing the natural neighbour interpolation 

In [360]:
fife_rainfall = fife_stations.copy()
rainfall = []
for i in fife_stations.index:
    j = list(sepa['station'][:]).index(i)
    rainfall.append(sepa['rainfall_amount'][1, j])
fife_rainfall['rainfall'] = rainfall

In [361]:
fife_rainfall

,name,longitude,latitude,geometry,rainfall
id,,,,,
15198,Annfield,-3.382615,56.059090,POINT (-3.38262 56.05909),79.0
501947,Baintown Rain-gauge,-3.049613,56.215117,POINT (-3.04961 56.21512),82.2
338380,Cambo Sands,-2.664088,56.298649,POINT (-2.66409 56.29865),52.2
15151,Fife Airport,-3.225782,56.177556,POINT (-3.22578 56.17756),95.6
473550,Kinghorn Ecology Centre Rain Gauge,-3.206299,56.069925,POINT (-3.20630 56.06992),71.6
335617,Kirkcaldy,-3.159471,56.115300,POINT (-3.15947 56.11530),82.4
15083,Newton of Falkland,-3.195810,56.249730,POINT (-3.19581 56.24973),87.4
15070,Rossie Farm,-3.213366,56.294485,POINT (-3.21337 56.29448),77.8
15155,Saline,-3.593886,56.119442,POINT (-3.59389 56.11944),88.0


In [362]:
# find the boundaries of the region of interest
x_min, y_min, x_max, y_max = fife_boundary.bounds

# how many kilometre squares is that?
xg_min, yg_min = proj_inv.transform(y_min, x_min)
xg_max, yg_max = proj_inv.transform(y_max, x_max)
es = int((xg_max - xg_min) / 1000) + 1
ns = int((yg_max - yg_min) / 1000) + 1

xs = numpy.linspace(x_min, x_max, num=es, endpoint=True)
ys = numpy.linspace(y_min, y_max, num=ns, endpoint=True)

In [364]:
%%timeit -n1 -r1

global df_voronoi, df_grid, tensor
df_voronoi = nnn_voronoi(fife_rainfall, fife_boundary)
df_grid = nnn_geometry(fife_rainfall, df_voronoi, xs, ys)
tensor = nnn_tensor_seq(fife_rainfall, df_voronoi, df_grid)

KeyError: 4

In [37]:
rr = numpy.array(fife_rainfall['rainfall'])
fife_interpolated_rainfall = apply_tensor(tensor, rr)

In [38]:
#fife_grid = nnn_masked_grid(fife_interpolated_rainfall, fife_boundary, xs, ys)

mask = numpy.empty(fife_interpolated_rainfall.shape)
for i in range(len(xs)):
    for j in range(len(ys)):
        x, y = xs[i], ys[j]
        mask[i, j] = not boundary_shape.contains(Point(x, y))

# return the masked grid
fife_grid =  numpy.ma.masked_where(mask, fife_interpolated_rainfall, copy=False)

We extract the interpolated rainfall into list form and normalise it to the same scale as the reference dataset.

In [39]:
# break-out lon and lat
rainpoints = []
mask = fife_grid.mask
for _, r in df_grid.iterrows():
    lon, lat = list(r.geometry.coords)[0]
    x, y = r.x, r.y
    if not mask[x, y]:
        rainpoints.append([lat, lon, (fife_grid[x, y] - fife_ref_min) * fife_ref_step])

In [40]:
fife_stations_interpolation = folium.Map(location=mid_fife, tiles="Stamen Terrain", zoom_start=10)
folium.GeoJson(fife['geometry']).add_to(fife_stations_interpolation)

# add the stations
for i, s in fife_stations.iterrows():
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    j = list(sepa['station'][:]).index(id)
    rain = sepa['rainfall_amount'][1, j]
    folium.Marker(location=(lat, lon),
                  tooltip=f'{i}: {name} ({lat:.2f}N, {lon:.2f}W) {rain:.2f}mm').add_to(fife_stations_interpolation)

# add the Voronoi cell boundaries
for c in fife_cells['geometry']:
    folium.GeoJson(c, style_function=lambda f: {'fill': False}).add_to(fife_stations_interpolation)
    
# add the heat map
_ = folium.plugins.HeatMap(data=rainpoints, min_opacity=0.01, radius=40, blur=40).add_to(fife_stations_interpolation)

In [41]:
fife_stations_interpolation

In [42]:
uk_ceh

In [43]:
len(rainpoints), len(fife_points)

(1287, 1330)

## Computing the tensor for the whole UK 

To compute the interpolation tensor for the whole UK we make use of the CEDA MIDAS network of stations.

In [5]:
ceda_filename='datasets/ceda_monthly_2017.nc'
ceda = nc.Dataset(ceda_filename)

We filter-out any stations not in Great Britain, so excluding Northern Ireland. 

In [6]:
gb_boundary = cascaded_union(counties.geometry)

In [7]:
cedastations = GeoDataFrame(columns=['name', 'id', 'longitude', 'latitude', 'geometry'])
for i in range(len(ceda['station'])):
    lat, lon = proj.transform(float(ceda['x'][i]), float(ceda['y'][i]))
    if Point(lon, lat).within(gb_boundary):
        cedastations.loc[i] = {'id': int(ceda['station'][i]),
                               'name': ceda['name'][i],
                               'longitude': lon,
                               'latitude': lat,
                               'geometry': Point(lon, lat)}
cedastations.set_index('id', inplace=True)

In [10]:
uk_gauges = folium.Map(location=(55, -3), tiles="Stamen Terrain", zoom_start=6)

# add the boundaries
county_boundaries_layer = folium.FeatureGroup(name='County boundaries')
for _, r in counties.iterrows():
    folium.GeoJson(r['geometry']).add_to(county_boundaries_layer)
county_boundaries_layer.add_to(uk_gauges)

# add the CEDA stations
ceda_layer = folium.FeatureGroup(name='CEDA MIDAS stations')
for i in cedastations.index:
    s = cedastations.loc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{i}: {name} ({lat:.2f}N, {lon:.2f}W)',
                  icon=folium.Icon(color='green', icon='cloud')).add_to(ceda_layer)
ceda_layer.add_to(uk_gauges)
    
# add a layer countrol
_ = folium.LayerControl().add_to(uk_gauges)

In [11]:
uk_gauges

In [8]:
# find the boundaries of the region of interest
x_min, y_min, x_max, y_max = gb_boundary.bounds

# how many kilometre squares is that?
xg_min, yg_min = proj_inv.transform(y_min, x_min)
xg_max, yg_max = proj_inv.transform(y_max, x_max)
es = int((xg_max - xg_min) / 1000) + 1
ns = int((yg_max - yg_min) / 1000) + 1

xs = numpy.linspace(x_min, x_max, num=es, endpoint=True)
ys = numpy.linspace(y_min, y_max, num=ns, endpoint=True)

In [9]:
%%timeit -n1 -r1

global gb_voronoi
gb_voronoi = nnn_voronoi(cedastations, gb_boundary)

2.03 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%%timeit -n1 -r1

global gb_grid
gb_grid = nnn_geometry(cedastations, gb_voronoi, xs, ys)

12min 11s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
with open('datasets/gb_voronoi.pickle', 'wb') as fh:
    pickle.dump(gb_voronoi, fh)
with open('datasets/gb_grid.pickle', 'wb') as fh:
    pickle.dump(gb_grid, fh)

In [98]:
with open('datasets/gb_voronoi.pickle', 'rb') as fh:
    gb_voronoi = pickle.load(fh)
with open('datasets/gb_grid.pickle', 'rb') as fh:
    gb_grid = pickle.load(fh)

In [15]:
gb_stations_voronoi = folium.Map(location=(55, -3), tiles="Stamen Terrain", zoom_start=6)

# add the stations
for i in cedastations.index:
    s = cedastations.loc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{i}: {name} ({lat:.2f}N, {lon:.2f}W)',
                  icon=folium.Icon(color='green', icon='cloud')).add_to(gb_stations_voronoi)
    
# add the Voronoi cells
cell_cmap = ['red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'black']
def cmap(i):
    return lambda c: {'fillColor': cell_cmap[i % len(cell_cmap)]}
for i in gb_voronoi.index:
    c = gb_voronoi.loc[i]['geometry']
    folium.GeoJson(c, style_function=cmap(i)).add_to(gb_stations_voronoi)

In [16]:
gb_stations_voronoi

In [17]:
def stationForCell(df_sample, df_voronoi, cell):
    return str(df_sample.loc[cell]['name']).strip()

In [65]:
def nnn_tensor_seq(df_points, df_voronoi, df_grid):
    '''Construct the natural nearest neighbour interpolation tensor from a
    set of samples taken within a boundary and sampled at the given
    grid of interpolation points.

    The tensor is a three-dimensional sparse `numpy.array` with axes
    corresponding to xs, ys, and points, with entries containing the
    weight given to each sample in interpolating each point.

    :param df_points: the sample points
    :param df_voronoi: the Voronoi diagram of the samples
    :param df_grid: the grid to interpolate onto
    :returns: a tensor'''

    # construct the tensor
    tensor = numpy.zeros((max(df_grid.x) + 1, max(df_grid.y) + 1, len(df_points)))

    # group the grid points by the real cell they lie within
    grid_grouped = df_grid.groupby('cell').groups

    # ignore any points outside the Voronoi diagram
    if -1 in grid_grouped.keys():
        del grid_grouped[-1]

    # construct the weights
    for real_cell in [9280]: #grid_grouped.keys():
        print(real_cell, stationForCell(df_points, df_voronoi, real_cell))
        # extract the neighbourhood of Voronoi cells,
        # the only ones that the cell around this sample point
        # can intersect and so the only computation we need to do
        df_real_neighbourhood = df_voronoi.loc[df_voronoi.loc[real_cell].neighbourhood]
        real_coords = points_to_coords(df_real_neighbourhood.centre)
        real_boundary_shape = df_voronoi.loc[real_cell].boundary

        # construct an array that will hold the co-ordinates of all the real points
        # and the synthetic point
        synthetic_coords = numpy.append(real_coords, [[0, 0]], axis=0)

        for pt in grid_grouped[real_cell]:
            # re-compute the Voronoi cells given the synthetic point
            p = df_grid.loc[pt]
            synthetic_coords[-1] = points_to_coords([p.geometry])[0]
            synthetic_voronoi_cells, synthetic_voronoi_centres = voronoi_regions_from_coords(synthetic_coords, real_boundary_shape)

            # get the synthetic cell
            synth = [i for i in synthetic_voronoi_centres.keys() if len(synthetic_coords) - 1 in synthetic_voronoi_centres[i]]
            if synth == []:
                print('Skipped')
                continue
            i = synth[0]
            synthetic_cell = synthetic_voronoi_cells[i]

            # get the index of the sample
            s = list(df_points.index).index(real_cell)
            
            # compute the weights
            synthetic_cell_area = synthetic_cell.area
            for _, r in df_real_neighbourhood.iterrows():
                area = r.geometry.intersection(synthetic_cell).area
                if area > 0.0:
                    tensor[int(p['x']), int(p['y']), s] = area / synthetic_cell_area

    return tensor

In [66]:
%%timeit -n1 -r1

global gb_tensor
gb_tensor = nnn_tensor_seq(cedastations, gb_voronoi, gb_grid)

9280 westonzoyland
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
55.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [67]:
gb_rainfall = cedastations.copy()
rainfall = []
for id in cedastations.index:
    j = list(ceda['station'][:]).index(id)
    rainfall.append(ceda['rainfall_amount'][1, j])
gb_rainfall['rainfall'] = rainfall

In [68]:
gb_rainfall

,name,longitude,latitude,geometry,rainfall
id,,,,,
38,achfary,-4.920076,58.306182,POINT (-4.92008 58.30618),171.1
8231,huntsham,-3.439265,50.952553,POINT (-3.43927 50.95255),94.9
50,strathy-east,-3.995386,58.557531,POINT (-3.99539 58.55753),62.7
60,poolewe,-5.599708,57.767447,POINT (-5.59971 57.76745),99.9
64,plockton,-5.656926,57.334277,POINT (-5.65693 57.33428),113.7
...,...,...,...,...,...
24219,mannington-hall,1.176164,52.842971,POINT (1.17616 52.84297),53.0
57118,bute-rothesay-no2,-5.066457,55.821956,POINT (-5.06646 55.82196),12.0
57233,margam-no-2,-3.732128,51.551088,POINT (-3.73213 51.55109),88.6


In [69]:
%%timeit -n1 -r1

global rr, gb_interpolated_rainfall
rr = numpy.array(gb_rainfall['rainfall'])
gb_interpolated_rainfall = apply_tensor_fast(gb_tensor, rr)

987 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [70]:
# break-out lon and lat
rainpoints = []
for _, r in gb_grid.iterrows():
    rf = gb_interpolated_rainfall[r.x, r.y]
    if rf > 0:
        lon, lat = list(r.geometry.coords)[0]
        rainpoints.append([lat, lon, rf])

In [71]:
gb_interpolation = folium.Map(location=(55, -3), tiles="Stamen Terrain", zoom_start=6)

# add the stations and their observations
for i in range(len(cedastations)):
    s = cedastations.iloc[i]
    name, lon, lat = s['name'], s['longitude'], s['latitude']
    rf = gb_rainfall.iloc[i]['rainfall']
    folium.Marker(location=(lat, lon),
                  tooltip=f'{name} ({lat:.2f}N, {lon:.2f}W) {rf:.2f}mm',
                  icon=folium.Icon(color='green', icon='cloud')).add_to(gb_interpolation)

# add the Voronoi cells
for i in range(len(gb_voronoi)):
    c = gb_voronoi.iloc[i]['geometry']
    folium.GeoJson(c).add_to(gb_interpolation)
    
# add the heat map
_ = folium.plugins.HeatMap(data=rainpoints, min_opacity=0.01, radius=40, blur=40).add_to(gb_interpolation)

In [72]:
gb_interpolation